## Multiple regression for object-selectivity
This code runs two different multiple linear regressions to object BOY and object EGG.

In each regression model, control stimulus whose target location was the same with the object was included to dissociate choice-response information.

In a regression model for object BOY (i.e, Boy and 5kHz were associated and required left choice response), 3 different regressors (i.e. Boy, 5kHz, interaction) were included and the model was applied to 4 different stimulus conditions (i.e., BOY-M, BOY-V, BOY-A, Control-Left).

The purpose of this analysis is to define object-selectivity using beta coefficients of each regressor and also detach choice-response coding from object coding.

In [6]:
import os
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks

from sklearn.linear_model import LinearRegression

import matplotlib as mpl
import matplotlib.pyplot as plt  
import seaborn as sns

from datetime import date
import time

import random

In [7]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [8]:
mother_path = 'D:/Multi-modal project/'

### Parameter setting

In [9]:
num_shuffle = 1000

gauss_sigma = 2

# colors for multimodal, vis-only, aud-only conditions
color = ['mediumorchid','cornflowerblue','lightcoral','gray']
color2 = ['cyan','magenta','brown']
linestyle = ['-',':']

today = str(date.today())

### Data preparation

In [10]:
unit_summary = pd.read_table(mother_path+'/analysis/result'+
                             '/1. Cluster summary/clusterSummary.csv',sep=',')
save_path = mother_path+'analysis/result/3. Multiple regression for object-selectivity/'
data_path = mother_path+'analysis/result/zFR export/13-Apr-2022 (5 trials)/'
os.chdir(data_path)

In [11]:
# empty list for result csv file
result=[]

In [12]:
cell_list = os.listdir(os.curdir)

# Data analysis

In [14]:
beta_visual_result = np.empty((0,96))
beta_auditory_result = np.empty((0,96))
beta_interaction_result = np.empty((0,96))

In [253]:
def plot_SDF_beta(df,linewidth,smooth,save,save_format):
    cond = [(df.Type=='Multimodal')&(df.RWD_Loc==boy_goal),
            (df.Type=='Multimodal')&(df.RWD_Loc==egg_goal),
            (df.Type=='Visual')&(df.RWD_Loc==boy_goal),
            (df.Type=='Visual')&(df.RWD_Loc==egg_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==boy_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==egg_goal),
            (df.Type=='Elemental')&(df.RWD_Loc==boy_goal),
            (df.Type=='Elemental')&(df.RWD_Loc==egg_goal)]
    
    fr_mean = np.zeros((len(cond),95))
    fr_sem = np.zeros((len(cond),95))
    for i in range(len(cond)):
        fr_mean[i,:] = df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy().mean(axis=0)
        fr_sem[i,:] = stats.sem(df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy())
    
    if smooth:
        for i in range(len(cond)):
            fr_mean[i,:] = gaussian_filter(fr_mean[i,:],sigma=gauss_sigma)
            fr_sem[i,:] = gaussian_filter(fr_sem[i,:],sigma=gauss_sigma)
            
    y_max = np.ceil(np.max(fr_mean+fr_sem))
    y_min = np.ceil(np.abs(np.min(fr_mean-fr_sem)))*-1
    
    fig,ax = plt.subplots(3,3,figsize=(15,10))
    plt.suptitle(cell_name.strip('.csv'),fontsize=15);
    x = np.arange(95)*10
    
    for i in range(len(cond)):
        if i%2==0:
            ls = linestyle[0]
        else:
            ls = linestyle[1]            
        ax[0,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[0,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        ax[i%2+1,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[i%2+1,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        if i<6:
            ax[int(np.floor(i/2)),1].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
            ax[int(np.floor(i/2)),1].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
            
    for i in range(6):
        ax[i%3,int(np.floor(i/3))].set_yticks(np.arange(y_min,y_max+0.1,1))
        ax[i%3,int(np.floor(i/3))].set_ylabel('z-scored FR',fontsize=13)
        ax[i%3,int(np.floor(i/3))].set_xlabel('Time (ms)',fontsize=13)  
        ax[i%3,int(np.floor(i/3))].set_xticks([0,400,950])
        ax[i%3,int(np.floor(i/3))].set_xlim([0,950])
    
    t2 = ['Visual term','Auditory term','Interaction term']
    c2 = ['blue','red','green']
    for i in range(3):
        ax[i,2].plot(x,beta_boy[i,:],color=c2[i],linestyle='-')
        ax[i,2].plot(x,beta_egg[i,:],color=c2[i],linestyle=':')
        ax[i,2].set_title(t2[i],fontsize=13)
        ax[i,2].set_yticks(np.arange(-1,1.1,0.5))    
        ax[i,2].set_ylabel('Beta coefficient',fontsize=13)
        ax[i,2].set_xlabel('Time (ms)',fontsize=13)    
        ax[i,2].set_xticks([0,400,950])
        ax[i,2].set_xlim([0,950])

    plt.tight_layout()
    
    if save:          
        fig_path = save_path+today+'/'+region+'/'
        if os.path.exists(fig_path) is False:
            os.makedirs(fig_path)    
        if save_format=='png':
            plt.savefig(fig_path+cell_name.strip('.csv')+'.png',dpi=100,facecolor='white')
        elif save_format=='svg':
            plt.savefig(fig_path+cell_name.strip('.csv')+'.svg')
        plt.close()

In [254]:
%%time
for cell_run,cell_name in enumerate(cell_list):
    loop_start = time.time()
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id = int(cell_info[0])
    rat_id = cell_info[1]
    session_id = cell_info[2]
    region = cell_info[5]    
    
    df = pd.read_csv(data_path+cell_name)
    df.drop(df[df.Correctness==0].index,inplace=True)
    df.reset_index(inplace=True,drop=True)
    
    boy_goal = df.loc[df['Visual']=='Boy','RWD_Loc'].values[0]
    boy_aud = df.loc[df['RWD_Loc']==boy_goal,'Auditory'].values[0]
    
    egg_goal = df.loc[df['Visual']=='Egg','RWD_Loc'].values[0]
    egg_aud = df.loc[df['RWD_Loc']==egg_goal,'Auditory'].values[0]    
    
    df_boy = df[df['RWD_Loc']==boy_goal].reset_index(drop=True)
    df_egg = df[df['RWD_Loc']==egg_goal].reset_index(drop=True)
    
    # multiple regression for Boy object
    df_boy.loc[df_boy['Visual']=='Boy','Visual'] = 1
    df_boy.loc[df_boy['Visual']!=1,'Visual'] = 0
    df_boy.loc[df_boy['Auditory']==boy_aud,'Auditory'] = 1
    df_boy.loc[df_boy['Auditory']!=1,'Auditory'] = 0
    df_boy['Interaction'] = df_boy['Visual']*df_boy['Auditory']
    x_boy = df_boy[['Visual','Auditory','Interaction']]
    
    # multiple regression for Egg object
    df_egg.loc[df_egg['Visual']=='Egg','Visual'] = 1
    df_egg.loc[df_egg['Visual']!=1,'Visual'] = 0
    df_egg.loc[df_egg['Auditory']==egg_aud,'Auditory'] = 1
    df_egg.loc[df_egg['Auditory']!=1,'Auditory'] = 0
    df_egg['Interaction'] = df_egg['Visual']*df_egg['Auditory']
    x_egg = df_egg[['Visual','Auditory','Interaction']]

    # get beta coefficient in multiple regression
    beta_boy = np.zeros((3,95))
    beta_egg = np.zeros((3,95))    
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    for t in range(95):
        y_boy = df_boy.iloc[:,fr_id+t]
        y_egg = df_egg.iloc[:,fr_id+t]
    
        lr_boy = LinearRegression()
        lr_boy.fit(x_boy,y_boy)
        beta_boy[:,t] = lr_boy.coef_ 
    
        lr_egg = LinearRegression()
        lr_egg.fit(x_egg,y_egg)
        beta_egg[:,t] = lr_egg.coef_ 
        
    # get shuffled beta coefficient of each timepoint using Multiple Regression
    #fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    #shuffled_beta = np.zeros((3,95,num_shuffle))
    #for s in range(num_shuffle):
    #    x_shuffle = np.random.permutation(x)
    #    for t_s in range(95):
    #        y = df.iloc[:,fr_id+t_s]
    #        lr = LinearRegression()
    #        lr.fit(x_shuffle,y)
    #        shuffled_beta[:,t_s,s] = lr.coef_        
            
    plot_SDF_beta(df,2,1,1,'png')
            
    loop_end = time.time()
    loop_time = divmod(loop_end-loop_start,60)
    print(cell_name.strip('.csv'), f'////// {cell_run+1}/{len(cell_list)} completed  //////  {int(loop_time[0])} min {loop_time[1]:.2f} sec')

0003-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.1 ////// 1/888 completed  //////  0 min 1.41 sec
0004-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.2 ////// 2/888 completed  //////  0 min 1.03 sec
0005-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.3 ////// 3/888 completed  //////  0 min 1.01 sec
0006-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.1 ////// 4/888 completed  //////  0 min 1.01 sec
0007-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.2 ////// 5/888 completed  //////  0 min 1.29 sec
0008-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.3 ////// 6/888 completed  //////  0 min 1.00 sec
0010-600-1-1-Crossmodal-TeV-deep-(-6.96 mm)-TT6.2 ////// 7/888 completed  //////  0 min 0.97 sec
0011-600-1-1-Crossmodal-PER-superficial-(-6.48 mm)-TT7.1 ////// 8/888 completed  //////  0 min 1.04 sec
0013-600-1-1-Crossmodal-PER-deep-(-6.48 mm)-TT8.2 ////// 9/888 completed  //////  0 min 1.00 sec
0014-600-1-1-Crossmodal-PER-deep-(-6.48 mm)-TT8.3 ////// 10/888 completed  //////  0 min 1.06 sec
0015

0093-600-5-5-Crossmodal-POR-deep-(-7.56 mm)-TT24.2 ////// 82/888 completed  //////  0 min 0.94 sec
0094-600-5-5-Crossmodal-POR-deep-(-7.56 mm)-TT24.3 ////// 83/888 completed  //////  0 min 0.97 sec
0095-600-5-5-Crossmodal-POR-deep-(-7.56 mm)-TT24.4 ////// 84/888 completed  //////  0 min 1.04 sec
0096-600-5-5-Crossmodal-POR-deep-(-7.56 mm)-TT24.5 ////// 85/888 completed  //////  0 min 1.11 sec
0174-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.1 ////// 86/888 completed  //////  0 min 1.17 sec
0175-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.10 ////// 87/888 completed  //////  0 min 1.07 sec
0176-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.11 ////// 88/888 completed  //////  0 min 1.06 sec
0177-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.2 ////// 89/888 completed  //////  0 min 0.97 sec
0178-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.3 ////// 90/888 completed  //////  0 min 1.03 sec
0179-602-1-1-Crossmodal-PER-deep-(-5.28 mm)-TT6.4 ////// 91/888 completed  //////  0 min 1.00 sec
0181-602-1-1-C

0279-602-3-3-Crossmodal-PER-superficial-(-4.68 mm)-TT10.1 ////// 163/888 completed  //////  0 min 1.03 sec
0281-602-3-3-Crossmodal-PER-superficial-(-4.68 mm)-TT10.3 ////// 164/888 completed  //////  0 min 1.09 sec
0283-602-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT11.2 ////// 165/888 completed  //////  0 min 0.98 sec
0284-602-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT11.3 ////// 166/888 completed  //////  0 min 0.95 sec
0285-602-3-3-Crossmodal-PER-superficial-(-4.56 mm)-TT11.4 ////// 167/888 completed  //////  0 min 0.92 sec
0286-602-3-3-Crossmodal-PER-superficial-(-4.2 mm)-TT12.1 ////// 168/888 completed  //////  0 min 0.97 sec
0287-602-3-3-Crossmodal-PER-superficial-(-4.2 mm)-TT12.2 ////// 169/888 completed  //////  0 min 0.92 sec
0291-602-3-3-Crossmodal-PER-superficial-(-4.2 mm)-TT12.6 ////// 170/888 completed  //////  0 min 0.90 sec
0292-602-3-3-Crossmodal-PER-superficial-(-4.2 mm)-TT12.7 ////// 171/888 completed  //////  0 min 0.88 sec
0293-602-3-3-Crossmodal-PER-superficial-(

1023-640-2-2-Crossmodal-PER-deep-(-4.56 mm)-TT11.5 ////// 243/888 completed  //////  0 min 1.03 sec
1024-640-2-2-Crossmodal-PER-deep-(-4.56 mm)-TT11.6 ////// 244/888 completed  //////  0 min 1.25 sec
1025-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.1 ////// 245/888 completed  //////  0 min 1.43 sec
1026-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.2 ////// 246/888 completed  //////  0 min 1.11 sec
1027-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.3 ////// 247/888 completed  //////  0 min 0.89 sec
1028-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.4 ////// 248/888 completed  //////  0 min 1.32 sec
1029-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.5 ////// 249/888 completed  //////  0 min 1.11 sec
1030-640-2-2-Crossmodal-PER-superficial-(-4.56 mm)-TT13.6 ////// 250/888 completed  //////  0 min 1.60 sec
1033-640-2-2-Crossmodal-PER-superficial-(-4.68 mm)-TT16.1 ////// 251/888 completed  //////  0 min 1.21 sec
1034-640-2-2-Crossmodal-TeV-superficial-(-5.64 mm)-

1340-647-4-4-Crossmodal-PER-superficial-(-5.64 mm)-TT17.6 ////// 324/888 completed  //////  0 min 1.70 sec
1341-647-4-4-Crossmodal-PER-superficial-(-5.88 mm)-TT19.1 ////// 325/888 completed  //////  0 min 1.16 sec
1343-647-4-4-Crossmodal-PER-superficial-(-5.88 mm)-TT19.3 ////// 326/888 completed  //////  0 min 1.32 sec
1345-647-4-4-Crossmodal-PER-superficial-(-5.88 mm)-TT19.5 ////// 327/888 completed  //////  0 min 0.96 sec
1346-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.1 ////// 328/888 completed  //////  0 min 1.13 sec
1347-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.2 ////// 329/888 completed  //////  0 min 0.92 sec
1348-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.3 ////// 330/888 completed  //////  0 min 0.95 sec
1349-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.4 ////// 331/888 completed  //////  0 min 0.92 sec
1350-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.5 ////// 332/888 completed  //////  0 min 0.90 sec
1351-647-4-4-Crossmodal-TeV-deep-(-6.12 mm)-TT21.6 ////// 333/888 comple

...

1846-679-2-2-Crossmodal-Auditory-deep-(-5.4 mm)-TT19.6 ////// 563/888 completed  //////  0 min 0.94 sec
1848-679-2-2-Crossmodal-Auditory-deep-(-5.52 mm)-TT20.1 ////// 564/888 completed  //////  0 min 1.07 sec
1850-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.2 ////// 565/888 completed  //////  0 min 0.93 sec
1851-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.3 ////// 566/888 completed  //////  0 min 0.93 sec
1854-679-2-2-Crossmodal-Auditory-deep-(-5.64 mm)-TT21.6 ////// 567/888 completed  //////  0 min 0.90 sec
1855-679-2-2-Crossmodal-Auditory-deep-(-6.64 mm)-TT22.1 ////// 568/888 completed  //////  0 min 1.13 sec
1857-679-3-3-Crossmodal-Visual-deep-(-6.36 mm)-TT1.1 ////// 569/888 completed  //////  0 min 1.00 sec
1858-679-3-3-Crossmodal-Visual-deep-(-6.24 mm)-TT2.1 ////// 570/888 completed  //////  0 min 0.88 sec
1859-679-3-3-Crossmodal-Visual-deep-(-6.24 mm)-TT2.2 ////// 571/888 completed  //////  0 min 0.89 sec
1860-679-3-3-Crossmodal-Visual-deep-(-6.24 mm)-TT3.1 ////// 572/8

1947-679-5-5-Crossmodal-Visual-deep-(-4.44 mm)-TT14.8 ////// 643/888 completed  //////  0 min 0.89 sec
1948-679-5-5-Crossmodal-Visual-deep-(-4.44 mm)-TT14.9 ////// 644/888 completed  //////  0 min 0.93 sec
1949-679-5-5-Crossmodal-Visual-deep-(-4.68 mm)-TT15.1 ////// 645/888 completed  //////  0 min 0.90 sec
1950-679-5-5-Crossmodal-Visual-deep-(-4.68 mm)-TT15.2 ////// 646/888 completed  //////  0 min 0.98 sec
1951-679-5-5-Crossmodal-Visual-deep-(-4.56 mm)-TT16.1 ////// 647/888 completed  //////  0 min 0.96 sec
1952-679-5-5-Crossmodal-Auditory-deep-(-5.28 mm)-TT17.1 ////// 648/888 completed  //////  0 min 1.15 sec
1953-679-5-5-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.1 ////// 649/888 completed  //////  0 min 0.96 sec
1954-679-5-5-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.2 ////// 650/888 completed  //////  0 min 0.94 sec
1955-679-5-5-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.3 ////// 651/888 completed  //////  0 min 0.93 sec
1956-679-5-5-Crossmodal-Auditory-deep-(-5.4 mm)-TT18.4 ////// 652/88

2218-699-1-1-Crossmodal-Visual-superficial-(-6.36 mm)-TT23.1 ////// 723/888 completed  //////  0 min 0.90 sec
2219-699-1-1-Crossmodal-Visual-deep-(-6.48 mm)-TT24.1 ////// 724/888 completed  //////  0 min 0.93 sec
2220-699-2-2-Crossmodal-Visual-deep-(-6.72 mm)-TT2.1 ////// 725/888 completed  //////  0 min 0.91 sec
2221-699-2-2-Crossmodal-PER-deep-(-5.76 mm)-TT5.1 ////// 726/888 completed  //////  0 min 0.95 sec
2222-699-2-2-Crossmodal-PER-deep-(-5.52 mm)-TT7.1 ////// 727/888 completed  //////  0 min 1.21 sec
2223-699-2-2-Crossmodal-PER-deep-(-5.52 mm)-TT7.2 ////// 728/888 completed  //////  0 min 1.33 sec
2225-699-2-2-Crossmodal-PER-deep-(-5.52 mm)-TT8.1 ////// 729/888 completed  //////  0 min 1.70 sec
2226-699-2-2-Crossmodal-Tev-deep-(-4.8 mm)-TT10.1 ////// 730/888 completed  //////  0 min 1.81 sec
2227-699-2-2-Crossmodal-PER-deep-(-4.68 mm)-TT11.1 ////// 731/888 completed  //////  0 min 1.24 sec
2228-699-2-2-Crossmodal-Auditory-deep-(-4.56 mm)-TT12.1 ////// 732/888 completed  //////  

2309-699-4-4-Crossmodal-PER-deep-(-5.2 mm)-TT9.1 ////// 804/888 completed  //////  0 min 1.66 sec
2310-699-4-4-Crossmodal-PER-deep-(-5.2 mm)-TT9.2 ////// 805/888 completed  //////  0 min 1.28 sec
2311-699-4-4-Crossmodal-PER-deep-(-5.2 mm)-TT9.3 ////// 806/888 completed  //////  0 min 1.81 sec
2312-699-4-4-Crossmodal-PER-deep-(-4.8 mm)-TT10.1 ////// 807/888 completed  //////  0 min 1.67 sec
2313-699-4-4-Crossmodal-Auditory-deep-(-4.56 mm)-TT12.1 ////// 808/888 completed  //////  0 min 2.03 sec
2316-699-4-4-Crossmodal-Auditory-deep-(-4.68 mm)-TT13.1 ////// 809/888 completed  //////  0 min 1.88 sec
2317-699-4-4-Crossmodal-Auditory-deep-(-4.68 mm)-TT13.2 ////// 810/888 completed  //////  0 min 1.73 sec
2318-699-4-4-Crossmodal-Auditory-deep-(-4.68 mm)-TT13.3 ////// 811/888 completed  //////  0 min 1.37 sec
2319-699-4-4-Crossmodal-Auditory-deep-(-4.68 mm)-TT13.4 ////// 812/888 completed  //////  0 min 1.45 sec
2320-699-4-4-Crossmodal-Auditory-deep-(-4.68 mm)-TT13.5 ////// 813/888 completed  

2404-699-6-6-Crossmodal-PER-deep-(-5.28 mm)-TT17.7 ////// 884/888 completed  //////  0 min 1.78 sec
2405-699-6-6-Crossmodal-Visual-deep-(-6.12 mm)-TT21.1 ////// 885/888 completed  //////  0 min 1.43 sec
2406-699-6-6-Crossmodal-Visual-deep-(-6.12 mm)-TT21.2 ////// 886/888 completed  //////  0 min 1.07 sec
2407-699-6-6-Crossmodal-Visual-deep-(-6.12 mm)-TT21.3 ////// 887/888 completed  //////  0 min 0.94 sec
2409-699-6-6-Crossmodal-Visual-superficial-(-6.36 mm)-TT23.1 ////// 888/888 completed  //////  0 min 0.94 sec
CPU times: total: 27min 32s
Wall time: 20min 6s


### Export CSV file

In [16]:
print('END')

END
